In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_6282/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_6282/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_6_21_7,0.998096,0.974417,0.994786,0.977246,0.000793,0.015187,0.003308,0.002925,0.009951,0.028157,0.986196,0.029145,64.279815,99.309750,"Hidden Size=[4, 2], regularizer=0.02, learning..."
1,model_6_32_2,0.999622,0.974094,0.990093,0.999801,0.000157,0.015379,0.006286,0.000049,0.005031,0.012542,0.997261,0.012982,67.514756,102.544691,"Hidden Size=[4, 2], regularizer=0.02, learning..."
2,model_6_10_7,0.995896,0.972689,0.988206,0.996017,0.001709,0.016213,0.007483,0.001895,0.018889,0.041339,0.970245,0.042790,62.743823,97.773757,"Hidden Size=[4, 2], regularizer=0.02, learning..."
3,model_6_40_9,0.997935,0.972230,0.989535,0.997037,0.000860,0.016485,0.006640,0.001455,0.013258,0.029322,0.985029,0.030352,64.117621,99.147555,"Hidden Size=[4, 2], regularizer=0.02, learning..."
4,model_6_22_1,0.998322,0.971065,0.985517,0.999215,0.000699,0.017177,0.009190,0.000446,0.010983,0.026436,0.987831,0.027364,64.532063,99.561997,"Hidden Size=[4, 2], regularizer=0.02, learning..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1067,model_1_47_6,0.092773,-789.492886,-1652.507208,0.820931,0.377759,469.270725,1049.161618,0.033557,0.200950,0.614621,-1.192464,0.636193,35.946999,59.767354,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1068,model_1_41_6,0.617877,-799.141156,-1601.537935,0.356008,0.159112,474.998355,1016.821266,0.061667,0.172409,0.398888,0.076536,0.412888,37.676297,61.496652,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1069,model_1_41_3,0.599738,-800.898364,-1606.812393,0.129404,0.166665,476.041509,1020.167946,0.083366,0.176143,0.408246,0.032699,0.422575,37.583542,61.403897,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1070,model_1_12_5,0.707453,-1011.653984,-1904.821753,0.637765,0.121813,601.155149,1209.256920,0.216900,0.147070,0.349018,0.293011,0.361267,38.210532,62.030888,"Hidden Size=[4], regularizer=0.02, learning_ra..."
